In [53]:
import mysql.connector
import json
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
import re
import numpy as np

In [2]:
with open("../crawling/tokens.json") as f:
    passwords = json.load(f)

pwd=passwords["mysql_password"]

In [3]:
# Connect to MySQL
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password=pwd,
    database='stampcollection'
)

cursor = connection.cursor()

In [4]:
df = pd.read_csv('../crawling/contents_merged/all_contents_updated_2.csv')

C:\Users\Robin\AppData\Local\Temp\ipykernel_26876\2775233162.py:1: DtypeWarning: Columns (13,16,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../crawling/contents_merged/all_contents_updated_2.csv')


In [63]:
# get row where Date_of_Issue is 08//1932
df[df['Date_of_Issue'] == '08//1932']["Set_Url"][2364]

'https://www.stampworld.com/stamps/Aegean-Islands/Postage-stamps/g0072//'

In [72]:
# get row 2364
df.loc[2364]

Country                                               Aegean Islands
Category                                              Postage stamps
Year                                                            1932
Set_Url            https://www.stampworld.com/stamps/Aegean-Islan...
Set_Name           1932 No 6-14 with Inscription "OFFICINA CARTE ...
Set_ID                                                         37318
Set_Description                         August WM: 1 Perforation: 14
Number                                                            72
Type                                                              C1
Denomination                                                      5C
Color                                                    purple rose
Description                                                      NaN
Stamps_Issued                                                    NaN
Mint_Condition                                                  1.50
Unused                            

# Edit the df to fit in the db later

In [5]:
def parse_date(date_str):
    if pd.isna(date_str):  # Check for NaN values
        return None
    try:
        # Try to parse the date in the format %d/%m/%Y
        return datetime.strptime(date_str, '%d/%m/%Y').strftime('%Y-%m-%d')
    except ValueError:
        try:
            # Try to parse the date in the format %Y
            return datetime.strptime(date_str, '%Y').strftime('%Y-01-01')
        except ValueError:
            try:
                # Handle the format MM//YYYY by assuming the first day of the month
                return datetime.strptime(date_str, '%m//%Y').strftime('%Y-%m-%d')
            except ValueError:
                # Print the date string that couldn't be parsed if it's not NaN
                print(f"Couldn't parse: {date_str}")
                # Return None if the date cannot be parsed
                return None

# Convert the date format in the DataFrame
df['Date_of_Issue'] = df['Date_of_Issue'].apply(parse_date)

In [ ]:
# # Create a new table
# # drop table if exists
# cursor.execute("DROP TABLE IF EXISTS contents")
# create_table_query = """
# CREATE TABLE contents (
#     Country VARCHAR(255),
#     Category VARCHAR(255),
#     Year INT,
#     Set_Url VARCHAR(255),
#     Set_Name TEXT,
#     Set_ID VARCHAR(255),
#     Set_Description TEXT,
#     Number VARCHAR(255),
#     Type VARCHAR(10),
#     Denomination VARCHAR(255),
#     Color VARCHAR(255),
#     Description TEXT,
#     Stamps_Issued VARCHAR(255),
#     Mint_Condition VARCHAR(255),
#     Unused VARCHAR(255),
#     Used VARCHAR(255),
#     `Letter/FDC` VARCHAR(255),
#     Currency VARCHAR(10),
#     Image_URL VARCHAR(255),
#     Date_of_Issue DATE,
#     Perforations VARCHAR(255),
#     Sheet_Size VARCHAR(255),
#     Design VARCHAR(255),
#     Engraved_By VARCHAR(255),
#     Height_Width VARCHAR(255),
#     Themes TEXT,
#     Image_Accuracy INT
# )
# """
# cursor.execute(create_table_query)

# clear table
cursor.execute("TRUNCATE TABLE contents")

# Insert data into the table
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    insert_row_query = """
    INSERT INTO contents (Country, Category, Year, Set_Url, Set_Name, Set_ID, Set_Description, Number, Type, Denomination, Color, Description, Stamps_Issued, Mint_Condition, Unused, Used, `Letter/FDC`, Currency, Image_URL, Date_of_Issue, Perforations, Sheet_Size, Design, Engraved_By, Height_Width, Themes, Image_Accuracy)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_row_query, tuple(row))
    connection.commit()

# Close the connection
cursor.close()
connection.close()


# check if database if complete

In [9]:
# get number of rows in table
cursor.execute("SELECT COUNT(*) FROM contents")
print(cursor.fetchall())
print(df.shape[0])

[(1063560,)]
1063560


# Add some columns for easier access


## Add a column for perforaations

The column should contain float numers

In [10]:
# make copy of df
df_copy = df.copy()

In [14]:
# list all unique values in Perforation column
unique_perforations = df_copy['Perforations'].unique()
# print ALL unique values
for perforation in unique_perforations:
    print(perforation)

Imperforated
14¼
nan
11
13
14
11¾
12¾
13 x 13¼
13¼ x 13¾
13¾
13 x 12¾
12¼
14 on 3 sides
13½ x 12¾
13 x 14
14 on three sides
12¾ on three sides
13¾ x 13¼
12¾ x 14
14 x 12¾
12¾ x 13
14½ x 14¾
14¾ x 14½
14 x 14½
11¾ on three sides
13½ x 13¼
12¾ x 13¼
13¼ on 3 sides
14½ x 13¾
13¾ x 14¼
13¼
12½
13½
12¾ x 13½
12½ x 12¾
13½ x 13
Rouletted 8
14¾ x 14¼
14¼ x 14¾
10½
12½ x 13
Rouletted 11
13 top and bottom
Rouletted 10
14 x 13
13 horizontally
12
10
121.2
11½
11¾ and 11½
11¼
14½
13 x 13½
14½ and 13 x 13½
14½ x 14
14 x 13½
13 x 11½
13½ x 14
13½ and 11 x 11½
12 x 13½
12¼ x 12¾
12 x 12½
13 x 12½
11 or 12
Zigzag Rouledded 10
11 on 2 or 3 sides
11 x 12
11-12
12 or 12 x 11
11-11½
12-13½
11 & 12
13½ x 14¼
14¼ x 13½
13 x 12
11½ x 12
12 or imperforated
12 x 11
13½ x 12
1
11 x 10½
10½ x 11
12½ x 12
12 x 11½
12-12½
12½ x 12¼
12¼ x 12½
12 x 13
36
Rouletted
13¼ x 13
13 - 14½
14 x 15
14 x 13½ & 14 x 14½
13 x 14½
14 x 13¾
13¾ x 13
13¾ x 13½
14 x 14¼
14½ x 13
14¼ x 13¾
14¼ x 14
14¼ x 13
13¼ x 12¾
14¾
14 x 14¾
41

In [139]:
pattern6 = r"^\d+(¼|½|⅓|⅔|¼|¾)? ((v|V)ertically|(v|V)ertical)$"
bool(re.match(pattern6, "12 vertical"))

True

In [ ]:
pattern1 = r"^\d+(¼|½|⅓|⅔|¼|¾)?( )+x( )+\d+(¼|½|⅓|⅔|¼|¾)?$"
pattern2 = r"^\d+(¼|½|⅓|⅔|¼|¾)?$"

# for vertical, vertically horizontal, horizontally
# pattern3 = r"^\d+(¼|½|⅓|⅔|¼|¾)? horizontally$"
pattern3 = r"^\d+(¼|½|⅓|⅔|¼|¾)? ((h|H)orizontally|(h|H)orizontal)$"
pattern6 = r"^\d+(¼|½|⅓|⅔|¼|¾)? ((v|V)ertically|(v|V)ertical)$"

pattern4 = r"^\d+(¼|½|⅓|⅔|¼|¾)?( )?(,|:|;|and|&|or|-|х|X)( )?\d+(¼|½|⅓|⅔|¼|¾)?$"
pattern5 = r".*\d+(¼|½|⅓|⅔|¼|¾)? x \d+(¼|½|⅓|⅔|¼|¾)?.*"


horizontal = [] # first  number (width)
vertical   = [] # second number (height)
keyword    = [] # a keyword regarding the perforation

# set first true if after x and false if before x
def getNumberFromString(string, first = True):
    
    fraction_map = { 
        '½': 0.5, 
        '⅓': 1/3, 
        '⅔': 2/3, 
        '¼': 0.25, 
        '¾': 0.75 
    }

    # split at " " and get first or last depending on first
    string = string.split(' ')[0] if first else string.split(' ')[-1]

    # Remove any non-numeric characters
    fraction_str = re.sub(r'[^\d½⅓⅔¼¾]+', '', string)
    
    # Split the string into integer and fractional parts
    for frac in fraction_map:
        if frac in fraction_str:
            parts = fraction_str.split(frac)
            integer_part = int(parts[0]) if parts[0] else 0
            fractional_part = fraction_map[frac]
            return integer_part + fractional_part
    
    # If no fractional part is found, return the integer part
    return int(fraction_str)

# apply this for all elements in unique_perforations
for perforation in unique_perforations:
    keywords = []

    perforation = str(perforation)

    # replace some specific terms
    # replace 121.2 with 12½
    perforation = perforation.replace('133/5', '13½')
    perforation = perforation.replace('133/3', '13½')
    perforation = perforation.replace('13/¼', '13¼')
    perforation = perforation.replace('14/½', '14½')
    perforation = perforation.replace('121.2', '12½')
    perforation = perforation.replace('Imperfoarted', 'Imperforated')
    perforation = perforation.replace('Imperforted', 'Imperforated')
    perforation = perforation.replace('x14', 'x 14')
    perforation = perforation.replace('133/¤', '13¾')
    perforation = perforation.replace('121/12', '12')
    perforation = perforation.replace('131/ ', '13½ ')
    perforation = perforation.replace('13½n', '13½')
    perforation = perforation.replace('13¾ x x13½', '13¾ x 13½')
    perforation = perforation.replace('top and bottom', 'horizontally')
    if(perforation == "¼ ½"):
        perforation = perforation.replace('¼ ½', '')

    # if perforation includes Imperforated make keyword Imperforated
    if 'Imperforated' in perforation or 'imperforated' in perforation or 'Imperforate' in perforation:
        keywords.append('Imperforated')
    if 'Printed' in perforation:
        keywords.append('Printed')
    if ' on 3 sides' in perforation or ' on three sides' in perforation:
        keywords.append('3 sides')
    if ' on 2 sides' in perforation or ' on two sides' in perforation:
        keywords.append('2 sides')
    if 'on 2 or 3 sides' in perforation:
        keywords.append('2 or 3 sides')
    if 'on 3 or 4 sides' in perforation:
        keywords.append('3 or 4 sides')
    if 'Zigzag Rouledded' in perforation:
        keywords.append('Zigzag Rouledded')
    if any(term in perforation.lower() for term in ['rouletted', 'roulette']):
        keywords.append('Rouletted')
    if perforation.lower() == "printed":
        keywords.append('Printed')
    if perforation.lower() == "different":
        keywords.append('Different')
    if perforation.lower() == "die cut" or perforation.lower() == "diecut":
        keywords.append('Die Cut')
    if perforation.lower() == "laser die cut":
        keywords.append('Laser Die Cut')
    if perforation.lower() == "laser-cut":
        keywords.append('Laser Cut')
    if perforation.lower() == "rough":
        keywords.append('Rough')
    if perforation.lower() == "wavy":
        keywords.append('Wavy')
    if perforation.lower() == "simulated":
        keywords.append('Simulated')
    if perforation.lower() == "cut":
        keywords.append('Cut')

    perforation = perforation.replace('11 or Rouletted', '11').strip()
    perforation = perforation.replace('/Printed', '').strip()

    # remove three sides
    perforation = perforation.replace(' on three sides', '').strip()
    perforation = perforation.replace(' on 3 sides', '').strip()
    perforation = perforation.replace(' on two sides', '').strip()
    perforation = perforation.replace(' on 2 sides', '').strip()
    perforation = perforation.replace(' on 2 or 3 sides', '').strip()
    perforation = perforation.replace(' on 3 or 4 sides', '').strip()
    # remove rouletted
    perforation = perforation.replace('rouletted', '').strip()
    perforation = perforation.replace('roulette', '').strip()
    perforation = perforation.replace('Rouletted', '').strip()
    perforation = perforation.replace('Roulette', '').strip()
    # remove imperforated
    perforation = perforation.replace('or imperforated', '').strip()
    perforation = perforation.replace('or Imperforated', '').strip()
    perforation = perforation.replace('Imperforated', '').strip()
    perforation = perforation.replace('imperforated', '').strip()
    perforation = perforation.replace('Imperforate', '').strip()

    # remove Zigzag Rouledded
    perforation = perforation.replace('Zigzag Rouledded', '').strip()
    # remove different
    perforation = perforation.replace('Different', '').strip()
    perforation = perforation.replace('different', '').strip()
    # remove die cut
    # remove laser die cut
    perforation = perforation.replace('Laser Die Cut', '').strip()
    perforation = perforation.replace('Laser-cut', '').strip()
    perforation = perforation.replace('Die Cut', '').strip()
    perforation = perforation.replace('Diecut', '').strip()
    perforation = perforation.replace('Cut', '').strip()
    perforation = perforation.replace('Rough', '').strip()
    perforation = perforation.replace('Wavy', '').strip()
    perforation = perforation.replace('Simulated', '').strip()

    if re.match(pattern3, perforation):
        keywords.append('Just Horizontal')
    if re.match(pattern6, perforation):
        keywords.append('Just Vertical')

    if re.match(pattern1, perforation):
        splitted = perforation.split(' x ')
        horizontal.append(getNumberFromString(splitted[0].strip(), first = False))
        vertical.append(getNumberFromString(splitted[1].strip(), first = True))
    elif re.match(pattern4, perforation):
        splitted = re.split(r'(:|,|;|and|&|or|-|х|X)', perforation)
        horizontal.append(getNumberFromString(splitted[0].strip(), first = False))
        vertical.append(getNumberFromString(splitted[2].strip(), first = True))
    elif re.match(pattern2, perforation):
        horizontal.append(getNumberFromString(perforation, first = True))
        vertical.append(getNumberFromString(perforation, first = True))
    elif re.match(pattern3, perforation):
        perforation = perforation.replace(' horizontally', '').strip()
        perforation = perforation.replace(' Horizontally', '').strip()
        perforation = perforation.replace(' horizontal', '').strip()
        perforation = perforation.replace(' Horizontal', '').strip()
        horizontal.append(getNumberFromString(perforation, first = True))
        vertical.append(None)
    elif re.match(pattern6, perforation):
        perforation = perforation.replace(' vertically', '').strip()
        perforation = perforation.replace(' Vertically', '').strip()
        perforation = perforation.replace(' vertical', '').strip()
        perforation = perforation.replace(' Vertical', '').strip()
        vertical.append(getNumberFromString(perforation, first = True))
        horizontal.append(None)
    
    elif re.match(pattern5, perforation):
        # should be last elif
        keywords.append('Multiple Perforations')
        # get first occurance of pattern
        splitted = re.findall(r'\d+(?:¼|½|⅓|⅔|¼|¾)? x \d+(?:¼|½|⅓|⅔|¼|¾)?', perforation)[0]
        splitted = re.split(r'(x|;|and|&|or|-|х)', splitted)
        horizontal.append(getNumberFromString(splitted[0].strip(), first = False))
        vertical.append(getNumberFromString(splitted[2].strip(), first = True))
    elif perforation.lower() == 'nan' or pd.isna(perforation) or perforation == '' or perforation.lower() == 'printed' or perforation.lower() == 'laser die cut':
        horizontal.append(None)
        vertical.append(None)
    else:
        horizontal.append("not implemented yet")
        vertical.append("not implemented yet")

    keyword.append(keywords)

# make a df of this and print the entire df
df_perforations = pd.DataFrame({'Perforations': unique_perforations, 'Horizontal': horizontal, 'Vertical': vertical, 'Keyword': keyword})
# Set display option to show all rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)  # Prevents wrapping of columns

print(df_perforations)

# Reset display option to default
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.expand_frame_repr')

# # loop over all values in the Perforations column
# for perforation in perfortions:
#     # check if the value matches the pattern
#     if re.match(pattern1, perforation):
#         splitted = perforation.split(' x ')
#         # convert the 

                                           Perforations           Horizontal             Vertical                                Keyword
0                                          Imperforated                 None                 None                         [Imperforated]
1                                                   14¼                14.25                14.25                                     []
2                                                   NaN                 None                 None                                     []
3                                                    11                   11                   11                                     []
4                                                    13                   13                   13                                     []
5                                                    14                   14                   14                                     []
6                                        

In [166]:
# get row where Perforations includes 131/
df_copy[df_copy['Perforations'].fillna('').str.contains('133/3')]["Set_Url"].iloc[0]

'https://www.stampworld.com/stamps/Argentina/Postage-stamps/g1480//'

In [157]:
print(splitted)

['14¼ ', '&', ' 12']


### merge dfs

In [158]:
# Merge the dataframes
df_merged = pd.merge(df_copy, df_perforations, on='Perforations', how='left')


In [159]:
all_horizontal = df_merged['Horizontal']

In [160]:
count_not_implemented = np.count_nonzero(all_horizontal == "not implemented yet")
count_none =  np.count_nonzero(all_horizontal.isnull())
print(count_not_implemented / len(all_horizontal))
print(count_none / len(all_horizontal))
print(1 - (count_not_implemented + count_none) / len(all_horizontal))

0.007724999059761556
0.27097954041144834
0.7212954605287901


In [57]:
all_horizontal

0          None
1          None
2          None
3          None
4          None
           ... 
1063555    None
1063556    None
1063557    None
1063558    None
1063559    None
Name: Horizontal, Length: 1063560, dtype: object